In [1]:
#%%

import os
import numpy as np
from smt.surrogate_models import SGP
from smt.sampling_methods import LHS
import scipy


function_ = 'function_4' 
xt = np.load(os.getcwd() + f'\\initial_data\\{function_}\\initial_inputs.npy')
yt = np.load(os.getcwd() + f'\\initial_data\\{function_}\\initial_outputs.npy')



if(function_ == 'function_1'):
    ndim = 2
if(function_ == 'function_4'):
    ndim = 4
if(function_ == 'function_8'):
    ndim = 8
max_obs=0      


 

sgp = SGP(method="VFE")  #Fully Independent Training Conditional (FITC) method or the Variational Free Energy (VFE)
sgp.set_training_values(xt, yt)
#sgp.set_inducing_inputs(Z=Z)
sgp.set_inducing_inputs()  # When Z not specified n_inducing points are picked randomly in traing data
sgp.train()

from smt.problems import TensorProduct
prob = TensorProduct(ndim=ndim, func="gaussian")
xlimits = prob.xlimits

if(function_ == 'function_1'):
        xlimits = np.array([[0.01, 0.99], [0.01, 0.99]])
        
if(function_ == 'function_4'):
        xlimits = np.array([[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0]])

if(function_ == 'function_8'):
    xlimits = np.array([[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0]])

 
samp = LHS(xlimits=xlimits, random_state=42, criterion='correlation' )
x = samp(10000)    
    
hat_y  = sgp.predict_values(x)
var = sgp.predict_variances(x)

    
acquisition_function = []
Z = (hat_y  - max_obs - 0) / np.sqrt(var)
probs = scipy.stats.norm.cdf(Z)
acquisition_function = np.array(probs)
    
idx_max = np.argmax(acquisition_function)
next_query = x[idx_max]

print(next_query)

___________________________________________________________________________
   
                                    SGP
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 30
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  0.2580180
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 10000
   
   Predicting ...
   Predicting - done. Time (sec):  0.0085030
   
   Prediction time/pt. (sec) :  0.0000009
   
[0.02680421 0.38434875 0.05086085 0.0713506 ]


In [ ]:
# %%
